In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import matplotlib.pyplot as plt
from functions_text import *

In [12]:
def importar_mesa():
    # Webscraping del texto de la estela de Mesa expuesto en el artículo de Wikipedia
    enlace ='https://en.wikipedia.org/wiki/Mesha_Stele'
    respuesta = requests.get(enlace)
    try:
        soup = bs(respuesta.content, 'html.parser')
        texto_por_linea = soup.find_all('span', class_="script-phoenician")
        mesa = pd.DataFrame({'Texto':[i.get_text() for i in texto_por_linea]})
        # Columna del número de línea en la estela
        mesa['línea'] = mesa.index+1
        return mesa
    except:
        # Si hay un error en la conexión, muestra el error
        print('Error en la conexión')
        print(respuesta)
        return 0
    
mesa = importar_mesa()

In [50]:
def palabras_mesa(df_lineas):
    '''
    df_lineas (pd.DataFrame): Requiere un dataframe de pandas con el texto de la estela por líneas.

    Función que devuelve un dataset con el texto por palabras. Corrige irregularidades en la primera línea.
    '''
    # Hay un error en la primera línea que voy a corregir a mano:
    palabras = df_lineas.loc[0,'Texto'].split()  
    print(palabras)
    palabras.pop(7)
    palabras.pop(7)
    print(palabras)

    linea_1 = [i for i in palabras if i != '𐤟']
    # Generacion del dataframe de palabras:
    diccionario_mesa = {'palabra':[], 'linea_estela':[], 'numero_oracion':[]}
    #línea 1:
    numero_oracion = 1
    for i in linea_1:
        diccionario_mesa['palabra'].append(i)
        diccionario_mesa['linea_estela'].append(1)
        diccionario_mesa['numero_oracion'].append(numero_oracion)

    # Resto de estela:
    for fila in range(1, len(df_lineas)-1):
        linea = [i for i in df_lineas.loc[fila, 'Texto'].split() if i != '𐤟']
        for  num, palabra in enumerate(linea):
            palabra = palabra.replace('[', '')
            palabra = palabra.replace(']', '')
            palabra = palabra.replace('.', '')
            palabra = palabra.replace('𐤟', '')
            # Unir primera palabra de la línea a la palabra anterior salvo en las líneas 19, 25 y 26
            if (fila not in [19, 25, 26]) and num == 0:
                diccionario_mesa['palabra'][-1] = diccionario_mesa['palabra'][-1] + palabra
            else:

                if palabra == '|':
                    numero_oracion += 1
                else:
                    
                    diccionario_mesa['palabra'].append(palabra)
                    diccionario_mesa['linea_estela'].append(fila+1)
                    diccionario_mesa['numero_oracion'].append(numero_oracion) 

    a = pd.DataFrame(diccionario_mesa)
    # Unir final de línea con inicio de línea. Voy a hacerlo hasta la línea 26, después ya está perdido el comienzo de línea
    a['transcripcion'] = a['palabra'].apply(feniciohebreo_a_latino)
    return a

In [49]:

palabras = palabras_mesa(mesa)
palabras[:20]

['𐤀𐤍𐤊', '𐤟', '𐤌𐤔𐤏', '𐤟', '𐤁𐤍', '𐤟', '𐤊𐤌𐤔', '?', '?', '𐤌𐤋𐤊', '𐤟', '𐤌𐤀𐤁', '𐤟', '𐤄𐤃']
['𐤀𐤍𐤊', '𐤟', '𐤌𐤔𐤏', '𐤟', '𐤁𐤍', '𐤟', '𐤊𐤌𐤔', '𐤌𐤋𐤊', '𐤟', '𐤌𐤀𐤁', '𐤟', '𐤄𐤃']


,palabra,linea_estela,numero_oracion,transcripcion
0,𐤀𐤍𐤊,1,1,Ank
1,𐤌𐤔𐤏,1,1,mš3
2,𐤁𐤍,1,1,bn
3,𐤊𐤌𐤔,1,1,kmš
4,𐤌𐤋𐤊,1,1,mlk
5,𐤌𐤀𐤁,1,1,mAb
6,𐤄𐤃𐤉𐤁𐤍𐤉,1,1,hdybny
7,𐤀𐤁𐤉,2,2,Aby
8,𐤌𐤋𐤊,2,2,mlk
9,𐤏𐤋,2,2,3l
